In [24]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

## Narrow breath hold length

## Load Files Function

In [25]:
def filelist(root):
    '''
    Return a fully-qualified list of filenames under root directory; 
    sort names alphabetically.
    '''
    allfiles = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            allfiles.append(os.path.join(path, name))
    return sorted(allfiles)

## Process Files (Helper Functions)

In [26]:
def longest_true_seq(bool_curve):
    '''
    Given an array of booleans,
    return indices of longest streak
    of Trues
    '''
    longest_streak = 0
    longest_streak_idx = []

    idx = [] 
    streak = 0
    for i in range(len(bool_curve)):
        if bool_curve[i] == True:
            streak += 1
            idx.append(i)
        else:
            if streak > longest_streak:
                longest_streak = streak
                longest_streak_idx = idx     
            streak = 0
            idx = []
    if streak > longest_streak and longest_streak < 150: # Not sure what this val should be
        return [0]
    return longest_streak_idx

In [27]:
def add_zeros(curve, bh_start_idx, bh_end_idx):
    ''' 
    Return trace with zeros appended to beginning 
    so length of input trace is 1500
    '''
    front_of_curve = curve[:bh_start_idx]
    num_zeros = 1400 - len(front_of_curve)
    zeros = np.zeros([1, num_zeros])[0]
    return np.concatenate((zeros, curve[:bh_end_idx]))

In [28]:
def process_curve(curve):
    '''
    Extract input trace from entire trace
    '''
    deriv = np.diff(curve)
    breath_hold_idx = longest_true_seq(abs(deriv)<=0.001) # Not sure what this val should be
    bh_start_idx = breath_hold_idx[0]
    
    if len(breath_hold_idx) < 100:
         return [], 0, 0

    bh_end_idx = breath_hold_idx[99] + 1
    if bh_start_idx < 1400:
        return add_zeros(curve, bh_start_idx, bh_end_idx), len(breath_hold_idx)*.01, breath_hold_idx
    
    else:
        curve_start_idx = bh_start_idx - 1400
        return curve[curve_start_idx:bh_end_idx], len(breath_hold_idx)*.01, breath_hold_idx
    

## Create DF of Inputs and Outputs

In [29]:
def get_inputs_outputs_df(file_root):
    '''
    Given a root with files, get a dataframe of shape (5680, 2)
    with input data traces (arrays) and 
    output data breath_holds (floats) 
    '''
    filenames = filelist(file_root)
    
    orig_curves = []
    csv_breath_holds = []
    traces = []
    data_breath_holds = []
    bh_idxs = []
    bh_start_end = []
    
    for file in filenames:
        if file.endswith('.CSV'):     
            df = pd.read_csv(file, header=None)
            curve = np.array(df.iloc[3:, 0], dtype='float32')
            input_trace, bh_len, bh_idx = process_curve(curve)
            if len(input_trace) != 0:
                orig_curves.append(curve)
                traces.append(input_trace)
                csv_breath_holds.append(float(df.iloc[1,1]))
                data_breath_holds.append(bh_len)
                bh_idxs.append(bh_idx)
                bh_start_end.append((bh_idx[0], bh_idx[-1]))
    
    data = {'Trace': traces,'Csv_breath_holds': csv_breath_holds, 'Data_breath_holds': data_breath_holds, 
            'Full_trace': orig_curves, "breathhold_idx": bh_idxs, 'bh_start_end':bh_start_end}
    return pd.DataFrame(data)

In [30]:
breath_df = get_inputs_outputs_df('data_sdx_')

In [31]:
breath_df.head()

,Trace,Csv_breath_holds,Data_breath_holds,Full_trace,breathhold_idx,bh_start_end
0,"[0.1319, 0.1363, 0.1408, 0.1452, 0.1497, 0.154...",31.29,31.69,"[0.0003, 0.0006, 0.001, 0.0014, 0.0019, 0.0023...","[2661, 2662, 2663, 2664, 2665, 2666, 2667, 266...","(2661, 5829)"
1,"[0.4727, 0.4687, 0.4646, 0.4605, 0.4563, 0.452...",30.61,31.40,"[0.0006, 0.0009, 0.0013, 0.0017, 0.0022, 0.002...","[2386, 2387, 2388, 2389, 2390, 2391, 2392, 239...","(2386, 5525)"
2,"[0.6466, 0.6496, 0.6526, 0.6555, 0.6583, 0.661...",14.39,18.43,"[0.0002, 0.0005, 0.0008, 0.0012, 0.0016, 0.002...","[2118, 2119, 2120, 2121, 2122, 2123, 2124, 212...","(2118, 3960)"
3,"[0.0982, 0.0966, 0.095, 0.0934, 0.0919, 0.0904...",30.85,31.84,"[0.0013, 0.0027, 0.0034, 0.0041, 0.005, 0.006,...","[2573, 2574, 2575, 2576, 2577, 2578, 2579, 258...","(2573, 5756)"
4,"[0.0117, 0.0109, 0.0102, 0.0095, 0.0089, 0.008...",32.12,32.53,"[0.0012, 0.0015, 0.0019, 0.0023, 0.0029, 0.003...","[2520, 2521, 2522, 2523, 2524, 2525, 2526, 252...","(2520, 5772)"


In [32]:
under2 = breath_df[abs(breath_df['Csv_breath_holds']-breath_df['Data_breath_holds'])<=2]

In [33]:
under2 = under2[(under2['Data_breath_holds']<=20)]

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
df = under2

In [35]:
x_train, x_test, y_train, y_test = train_test_split(df['Trace'], df['Data_breath_holds'], test_size=0.2)

In [36]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

In [37]:
# Change type of each row from np.array to list to put into scaler
x_train = np.array([list(row) for row in x_train])
x_val = np.array([list(row) for row in x_val])
x_test = np.array([list(row) for row in x_test])

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [38]:
scaler = StandardScaler() # creates the scaler
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

## Initialize Data for Model

In [39]:
#dataset
from torch.utils.data import Dataset

In [40]:
class timeseries(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len

In [41]:
dataset = timeseries(x_train,y_train)
valid_dataset = timeseries(x_val,y_val)
test_dataset = timeseries(x_test,y_test)

In [42]:
train_loader = DataLoader(dataset,shuffle=True,batch_size=100)
val_loader = DataLoader(valid_dataset,shuffle=False,batch_size=100)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=100)

## Define LSTM Model (One Step Prediction)

In [43]:
#neural network
from torch import nn

class LSTM_Model(nn.Module):
    def __init__(self):
        super(LSTM_Model,self).__init__()
        self.lstm = nn.LSTM(input_size=1,hidden_size=100,num_layers=1,batch_first=True) # change hidden size
        self.linear = nn.Linear(in_features=100,out_features=1)
        
        
    def forward(self,x):
        output,_status = self.lstm(x)
        output = output[:,-1,:]
        #output = self.linear(torch.relu(output))
        output = self.linear(output)
        return output

model = LSTM_Model()

In [44]:
# Declaring Criterion and Optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.1)
epochs = 50

## Training Loop

In [45]:
history = []
for i in range(epochs):
    
    train_mse = 0.0
    train_mae = 0.0
    train_loss = 0.0
    train_preds = []
    for x, y in train_loader:
        
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        ypred = model(x.view(-1,1500,1))       
        # Find the Loss
        loss = criterion(ypred,y.view(-1,1))      
        # Calculate gradients
        loss.backward()       
        # Update Weights
        optimizer.step()      
        # Calculate L1 Loss
        train_loss += loss.item()
        # Calculate MAE
        ypred = ypred.detach().numpy()
        train_mae += mean_absolute_error(ypred, y.view(-1,1))
        # Calculate MSE
        train_mse += mean_squared_error(ypred, y.view(-1,1))
        # R^2
        
        train_preds.append(ypred)
        
    valid_mse = 0.0
    valid_mae = 0.0
    valid_loss = 0.0
    val_preds = []
    for x, y in val_loader:
        # Forward Pass
        ypred = model(x.view(-1,1500,1))
        # Find the Loss
        loss = criterion(ypred,y.view(-1,1))
        # Calculate L1 Loss
        valid_loss += loss.item()
        # Calculate MAE
        ypred = ypred.detach().numpy()
        valid_mae += mean_absolute_error(ypred, y.view(-1,1))
        # Calculate MSE
        valid_mse += mean_squared_error(ypred, y.view(-1,1))
        
        val_preds.append(ypred)

    history.append((train_loss/len(train_loader),valid_loss/len(val_loader)))
    if i%2 == 0:
        torch.save(model, 'lstm_single_output')
        print(f'Epoch {i+1} \t Training Loss: {train_loss/len(train_loader)} \tValidation Loss: {valid_loss/len(val_loader)}')
        print(f'\t\t Training MAE: {train_mae/len(train_loader)} \tValidation MAE: {valid_mae/len(val_loader)}')
        print(f'\t\t Training MSE: {train_mse/len(train_loader)} \tValidation MSE: {valid_mse/len(val_loader)}')
        print()
    
    

Epoch 1 	 Training Loss: 92.54723834991455 	Validation Loss: 37.607330322265625
		 Training MAE: 7.635125160217285 	Validation MAE: 5.533816337585449
		 Training MSE: 92.54723644256592 	Validation MSE: 37.607330322265625

Epoch 3 	 Training Loss: 39.82405662536621 	Validation Loss: 37.57332229614258
		 Training MAE: 5.550691366195679 	Validation MAE: 5.515478134155273
		 Training MSE: 39.82405471801758 	Validation MSE: 37.57332229614258

Epoch 5 	 Training Loss: 38.40311908721924 	Validation Loss: 40.97291946411133
		 Training MAE: 5.396719336509705 	Validation MAE: 5.6146559715271
		 Training MSE: 38.403117179870605 	Validation MSE: 40.97291564941406

Epoch 7 	 Training Loss: 39.373531341552734 	Validation Loss: 38.71126174926758
		 Training MAE: 5.480738162994385 	Validation MAE: 5.596057891845703
		 Training MSE: 39.37353038787842 	Validation MSE: 38.71126174926758

Epoch 9 	 Training Loss: 41.89150619506836 	Validation Loss: 39.846229553222656
		 Training MAE: 5.791084289550781 	Va

## Plot Loss

In [ ]:
history = torch.tensor(history)

In [ ]:
#torch.save(model, 'lstm_bh_epochs')

In [ ]:
def plot_loss(history, ax=None, maxy=None, file=None):
    if ax is None:
        fig, ax = plt.subplots(1,1, figsize=(6,3))
    ax.set_ylabel("Loss")
    ax.set_xlabel("Epochs")
    loss = history[:,0]
    val_loss = history[:,1]

    ax.spines['top'].set_visible(False)    # turns off the top "spine" completely
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(.5)
    ax.spines['bottom'].set_linewidth(.5)
    ax.plot(loss, label='train_loss')
    ax.plot(val_loss, '--', label='val_loss')
    ax.legend(loc='upper right')
    plt.tight_layout()
    if file:
        plt.savefig(f"{file}.pdf")

In [ ]:
plot_loss(history, maxy = 50)

In [ ]:
#test set actual vs predicted
test_set = timeseries(x_test,y_test)
test_pred = model(test_set[:][0].view(-1,1500,1)).view(-1)
plt.scatter(test_set[:][1].view(-1), test_pred.detach().numpy(),label='predicted')
plt.show()

In [50]:
test_pred = model(test_set[:][0].view(-1,1500,1)).view(-1)

In [51]:
test_set[:][1].view(-1)

tensor([34.5000, 35.4700, 35.4200, 30.7400, 32.5900, 36.6200, 27.0900, 30.7700,
        39.9000, 27.1700, 56.7200, 29.1300, 33.1000, 27.2000, 55.6500, 12.7800,
        31.0900,  2.1300, 29.9400, 59.4200, 45.8600, 10.7100, 30.6700, 46.5500,
        29.9400, 36.9900, 30.5700, 38.6600, 36.1900, 25.6800, 31.1600, 29.9100,
        29.4100, 34.8500, 31.5000, 43.6200, 33.9100, 29.2400, 41.3500, 27.9700,
        32.8200, 30.5400, 31.5300, 34.6700, 39.3200, 41.0200, 34.2600, 40.2400,
        39.4200, 30.6700, 26.7400,  1.2700, 42.9900, 42.3600, 10.9300, 29.3100,
        26.9200, 46.3000, 50.6700, 29.5700, 38.3100, 32.8500, 36.8900, 41.7700,
        31.3700, 46.0500, 36.8400, 25.8100, 34.0800, 30.5800, 29.3100, 35.6300,
        32.2100, 33.9400, 38.4000, 41.6600, 52.6500, 30.9000, 36.3100,  1.5300,
        31.0000, 26.7800,  5.1700, 29.5800, 29.9500, 22.6600, 33.0200, 21.0000,
        42.2700,  1.8600, 26.1800, 30.0400, 36.3600, 34.3200, 58.8600, 35.4300,
        16.1700, 22.1900, 26.3200, 37.69

In [52]:
test_pred

tensor([31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206, 31.9206,
        31.9206, 31.9206, 31.9206, 31.92